In [ ]:

# 叶节点编码 + Top Features + **LassoCV** 租金预测 (Rent Prediction) - V3 (Log Target, Combined Features, External CV)

import pandas as pd
import numpy as np
import os
import joblib # 用于将来可能保存对象
from sklearn.model_selection import KFold, cross_val_score # 导入 cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoCV # 导入 LassoCV
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, median_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack, csr_matrix # 用于组合稀疏和稠密矩阵

print("开始执行 Rent 预测流程 (LassoCV)...")
print("=======================================")

# --- 1. 配置 ---
# 输入文件夹 (包含 Rent 的 _selected.csv 文件)
INPUT_FOLDER = 'Rent_Capped_Aggregated_Data'
# 输出文件夹 (存放本次 Rent 预测结果)
OUTPUT_FOLDER = 'LeafEncoding_Combined_Lasso_Prediction_Rent_Log' # 文件夹名区分 Lasso
# 输入文件名
TRAIN_RENT_FILE = 'train_rent_capped_agg.csv' # 使用 Rent 的筛选后文件
TEST_RENT_FILE = 'test_rent_capped_agg.csv'   # 使用 Rent 的筛选后文件
# 输出文件名
OUTPUT_FILE = 'submission_leaf_combined_lasso_rent_log_mae.csv' # 文件名区分 Lasso

TARGET_COLUMN = 'Price'
ID_COLUMN = 'ID'
RANDOM_STATE = 42

# GBRT 参数 (保持不变)
GBRT_PARAMS = {
    'n_estimators': 150,
    'max_depth': 4,
    'min_samples_leaf': 30,
    'learning_rate': 0.1,
    'subsample': 0.7,
    'random_state': RANDOM_STATE
}

# 保留多少个最重要的原始特征与叶节点结合
N_TOP_ORIGINAL_FEATURES = 10

# LassoCV 参数
# Lasso 的 Alpha 范围通常可以小一些，但叶节点多，也可以尝试较大的值
LASSO_ALPHAS = np.logspace(-5, 2, 10) # 例如: [1e-5, ..., 100]
LASSO_CV_FOLDS = 5 # LassoCV 内部交叉验证折数 (优化 MSE)
LASSO_MAX_ITER = 3000 # 增加迭代次数
LASSO_TOL = 1e-3 # 放宽收敛容忍度

# 外部交叉验证折数 (评估 MAE)
EXTERNAL_CV_FOLDS = 6

# --- 2. 创建输出文件夹 ---
try:
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"输出文件夹 '{OUTPUT_FOLDER}' 已创建或已存在。")
except OSError as e:
    print(f"创建文件夹 '{OUTPUT_FOLDER}' 时出错: {e}")
    exit()

# --- 3. 加载 Rent 数据 ---
print(f"\n--- 从 '{INPUT_FOLDER}' 加载 Rent 数据 ---")
try:
    train_rent_df = pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_RENT_FILE), encoding='utf-8-sig')
    test_rent_df = pd.read_csv(os.path.join(INPUT_FOLDER, TEST_RENT_FILE), encoding='utf-8-sig')
    print("Rent 数据加载成功。")
    print(f"  训练集形状: {train_rent_df.shape}")
    print(f"  测试集形状: {test_rent_df.shape}")
except FileNotFoundError as e:
    print(f"加载 Rent 数据时出错: {e}.")
    exit()
except Exception as e:
    print(f"加载 Rent 数据时发生其他错误: {e}")
    exit()

# --- 转换布尔列 ---
print("\n--- 转换布尔列 (Rent) 为整数 (1/0) ---")
# (省略了详细打印)
for df_name in ['train_rent_df', 'test_rent_df']:
    df_obj = globals()[df_name]
    if df_obj is None: continue
    bool_columns = df_obj.select_dtypes(include='bool').columns
    if not bool_columns.empty:
        print(f"  转换 {df_name} 中的 {len(bool_columns)} 个布尔列...")
        for col in bool_columns:
            df_obj.loc[:, col] = df_obj[col].astype(int)
print("--- 布尔列转换完成 ---")

# 存储测试集的 ID
test_rent_ids = test_rent_df[ID_COLUMN].copy()

# --- 4. 准备数据 (分离 X/y, Log Transform y, 对齐, 填充 NaN) ---
print("\n--- 准备 Rent 训练和测试数据 ---")
try:
    # !!! 应用 Log Transform !!!
    y_train_rent = np.log1p(train_rent_df[TARGET_COLUMN])
    print(f"  已对 Rent 目标变量 '{TARGET_COLUMN}' 应用 log1p 转换。")

    X_train_rent = train_rent_df.drop(columns=[TARGET_COLUMN])
    X_test_rent = test_rent_df.drop(columns=[ID_COLUMN])

    # 移除可能残余的非数值列
    non_numeric_train_rent = X_train_rent.select_dtypes(exclude=np.number).columns
    if not non_numeric_train_rent.empty:
        print(f"  警告: Rent 训练集发现非数值列，将移除: {non_numeric_train_rent.tolist()}")
        X_train_rent = X_train_rent.drop(columns=non_numeric_train_rent)
    non_numeric_test_rent = X_test_rent.select_dtypes(exclude=np.number).columns
    if not non_numeric_test_rent.empty:
         print(f"  警告: Rent 测试集发现非数值列，将移除: {non_numeric_test_rent.tolist()}")
         X_test_rent = X_test_rent.drop(columns=non_numeric_test_rent)

    # 对齐列
    train_rent_cols = X_train_rent.columns
    test_rent_cols = X_test_rent.columns
    missing_in_test_rent = set(train_rent_cols) - set(test_rent_cols)
    for c in missing_in_test_rent: X_test_rent[c] = 0
    extra_in_test_rent = set(test_rent_cols) - set(train_rent_cols)
    if extra_in_test_rent: X_test_rent = X_test_rent.drop(columns=list(extra_in_test_rent))
    X_test_rent = X_test_rent[train_rent_cols]

    print(f"  对齐后 Rent 特征形状: Train={X_train_rent.shape}, Test={X_test_rent.shape}")

    # **填充缺失值**
    print("  使用中位数填充缺失值...")
    imputer_rent = SimpleImputer(strategy='median')
    X_train_rent_imputed = imputer_rent.fit_transform(X_train_rent)
    X_test_rent_imputed = imputer_rent.transform(X_test_rent)
    print("  Rent 数据缺失值填充完成。")

except Exception as e:
     print(f"Rent 数据准备时发生错误: {e}")
     exit()

# --- 5. 训练 GBRT 模型 (Rent) 以生成叶节点 & 获取特征重要性 ---
print("\n--- 训练 Gradient Boosting 模型 (Rent) 以生成叶节点特征 ---")
gbrt_rent = GradientBoostingRegressor(**GBRT_PARAMS)
try:
    print(f"  使用以下参数训练 GBRT: {GBRT_PARAMS}")
    gbrt_rent.fit(X_train_rent_imputed, y_train_rent)
    print("  GBRT 模型 (Rent) 训练完成。")
    importances_rent = gbrt_rent.feature_importances_
    top_n_indices_rent = np.argsort(importances_rent)[::-1][:N_TOP_ORIGINAL_FEATURES]
    top_n_features_rent = train_rent_cols[top_n_indices_rent].tolist()
    print(f"  识别出 Rent Top {N_TOP_ORIGINAL_FEATURES} 原始特征: {top_n_features_rent}")
except Exception as e:
    print(f"  训练 GBRT (Rent) 或获取重要性时出错: {e}")
    top_n_features_rent = []

# --- 6. 获取叶节点索引 (Rent) ---
print("\n--- 获取叶节点索引 (Rent) ---")
try:
    train_rent_leaf_indices = gbrt_rent.apply(X_train_rent_imputed)
    test_rent_leaf_indices = gbrt_rent.apply(X_test_rent_imputed)
except Exception as e:
    print(f"  获取叶节点索引 (Rent) 时出错: {e}")
    exit()

# --- 7. 对叶节点索引进行 One-Hot 编码 (Rent) ---
print("\n--- 对叶节点索引进行 One-Hot 编码 (Rent) ---")
leaf_encoder_rent = OneHotEncoder(handle_unknown='ignore', )
try:
    X_train_rent_leaves_encoded = leaf_encoder_rent.fit_transform(train_rent_leaf_indices)
    X_test_rent_leaves_encoded = leaf_encoder_rent.transform(test_rent_leaf_indices)
    n_leaf_features_rent = X_train_rent_leaves_encoded.shape[1]
    print(f"  生成了 {n_leaf_features_rent} 个 Rent 叶节点二元特征。")
except Exception as e:
    print(f"  One-Hot 编码 (Rent) 时出错: {e}")
    exit()

# --- 8. 组合叶节点特征与 Top N 原始特征 (Rent) ---
print(f"\n--- 组合 Rent 叶节点特征与 Top {len(top_n_features_rent)} 原始特征 ---")
try:
    if top_n_features_rent:
        X_train_rent_top_features = X_train_rent_imputed[:, top_n_indices_rent]
        X_test_rent_top_features = X_test_rent_imputed[:, top_n_indices_rent]
        X_train_rent_final = hstack([X_train_rent_leaves_encoded, csr_matrix(X_train_rent_top_features)])
        X_test_rent_final = hstack([X_test_rent_leaves_encoded, csr_matrix(X_test_rent_top_features)])
        print(f"  组合后 Rent 最终特征形状: Train={X_train_rent_final.shape}, Test={X_test_rent_final.shape}")
    else:
        X_train_rent_final = X_train_rent_leaves_encoded
        X_test_rent_final = X_test_rent_leaves_encoded
        print("  只使用编码后的 Rent 叶节点特征。")
except Exception as e:
    print(f"  组合 Rent 特征时出错: {e}")
    exit()

# --- 9. 定义评价标准 (MAE Scorer) ---
print("\n--- 定义评价标准 (MAE) ---")
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
print("MAE 评价标准已创建 (得分越低越好)。")
# 辅助函数
def calculate_metrics(y_true_log, y_pred_log):
    rmse_log = np.sqrt(mean_squared_error(y_true_log, y_pred_log))
    mae_log = mean_absolute_error(y_true_log, y_pred_log)
    medae_log = median_absolute_error(y_true_log, y_pred_log)
    y_true_orig = np.expm1(y_true_log)
    y_pred_orig = np.expm1(y_pred_log)
    y_pred_orig = np.clip(y_pred_orig, a_min=0, a_max=None)
    rmse_orig = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    mae_orig = mean_absolute_error(y_true_orig, y_pred_orig)
    medae_orig = median_absolute_error(y_true_orig, y_pred_orig)
    return rmse_orig, mae_orig, medae_orig

# --- 10. 定义最终的 LassoCV Pipeline (Rent) ---
print("\n--- 定义最终的 LassoCV Pipeline (Rent) ---")

lasso_cv_model_rent = LassoCV( # 新实例
    alphas=LASSO_ALPHAS,       # 提供 alpha 搜索范围
    cv=LASSO_CV_FOLDS,         # LassoCV 内部交叉验证折数 (优化 MSE)
    random_state=RANDOM_STATE,
    n_jobs=-1,
    max_iter=LASSO_MAX_ITER,
    tol=LASSO_TOL
    # 注意: LassoCV 不接受 scoring 参数
)

# 构建最终的 Pipeline
final_pipeline_rent = Pipeline([ # 新 Pipeline 实例
    ('scaler', StandardScaler(with_mean=False)), # 稀疏矩阵不中心化
    ('model', lasso_cv_model_rent)            # 使用 LassoCV 模型
])

# --- 11. 外部交叉验证评估 Pipeline (Rent, 评估 MAE) ---
print(f"\n--- 使用 {EXTERNAL_CV_FOLDS}-折交叉验证评估最终 Rent Pipeline (MAE in log space) ---")
try:
    external_cv_rent = KFold(n_splits=EXTERNAL_CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    # 使用 mae_scorer 进行外部评估
    cv_scores_log_mae_rent = cross_val_score(final_pipeline_rent, X_train_rent_final, y_train_rent,
                                             cv=external_cv_rent, scoring=mae_scorer, n_jobs=-1)
    mean_cv_log_mae_rent = np.mean(cv_scores_log_mae_rent)
    std_cv_log_mae_rent = np.std(cv_scores_log_mae_rent)
    print(f"  Rent 外部交叉验证得分 (负 Log MAE): {mean_cv_log_mae_rent:.4f} +/- {std_cv_log_mae_rent:.4f}")
    print(f"  Rent 外部交叉验证得分 (Log MAE): {-mean_cv_log_mae_rent:.4f}")
except Exception as e:
    print(f"  Rent 外部交叉验证时出错: {e}")

# --- 12. 在全部 Rent 训练数据上训练最终 Pipeline ---
print("\n--- 在全部 Rent 训练数据上训练最终的 LassoCV Pipeline ---")
try:
    final_pipeline_rent.fit(X_train_rent_final, y_train_rent)
    print("  最终 Rent Pipeline (LassoCV) 训练完成。")
    best_alpha_rent = final_pipeline_rent.named_steps['model'].alpha_
    print(f"  LassoCV (Rent) 找到的最佳 alpha (优化 MSE): {best_alpha_rent:.6f}")
    # 查看系数稀疏性
    coefs_rent = final_pipeline_rent.named_steps['model'].coef_
    print(f"  Lasso 模型 (Rent) 使用了 {np.sum(coefs_rent != 0)} / {len(coefs_rent)} 个特征。")

except Exception as e:
    print(f"  训练最终 Rent Pipeline (LassoCV) 时出错: {e}")
    exit()

# --- 13. 在 Rent 训练集上评估最终模型 (原始价格空间) ---
print("\n--- 在完整 Rent 训练集上评估最终 Lasso 模型 (原始租金空间) ---")
try:
    y_train_pred_rent_log = final_pipeline_rent.predict(X_train_rent_final)
    rmse_train_rent, mae_train_rent, medae_train_rent = calculate_metrics(y_train_rent, y_train_pred_rent_log)
    print(f"  Rent 训练集 RMSE (原): {rmse_train_rent:.2f}")
    print(f"  Rent 训练集 MAE  (原): {mae_train_rent:.2f}")
    print(f"  Rent 训练集 MedAE(原): {medae_train_rent:.2f}")
except Exception as e:
    print(f"  评估 Rent 训练集时出错: {e}")

# --- 14. 在 Rent 测试集上进行预测 ---
print("\n--- 在 Rent 测试集上进行预测 ---")
try:
    predictions_rent_log = final_pipeline_rent.predict(X_test_rent_final)
    predictions_rent = np.expm1(predictions_rent_log)
    print("  Rent 预测完成并已转换回原始租金空间。")
    if np.any(predictions_rent < 0):
        print(f"  警告: 发现 {np.sum(predictions_rent < 0)} 个负数预测值，将修正为 0。")
        predictions_rent = np.clip(predictions_rent, a_min=0, a_max=None)
except Exception as e:
    print(f"  在 Rent 测试集上预测时出错: {e}")
    exit()

# --- 15. 创建并保存 Rent 提交文件 ---
print(f"\n--- 创建并保存 Rent 提交文件到 '{OUTPUT_FOLDER}' ---")
submission_df_rent = pd.DataFrame({
    ID_COLUMN: test_rent_ids,
    'PredictedPrice': predictions_rent
})
submission_df_rent[ID_COLUMN] = submission_df_rent[ID_COLUMN].astype(int)
submission_df_rent = submission_df_rent.sort_values(by=ID_COLUMN)
try:
    output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILE)
    submission_df_rent.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✓ Rent 提交文件已成功保存: {output_path}")
    print(f"  总预测条数: {len(submission_df_rent)}")
    print("\nRent 提交文件预览 (前5行):")
    print(submission_df_rent.head())
except Exception as e:
    print(f"✗ 保存 Rent 提交文件时出错: {e}")

print("\n--- Rent 叶节点编码 + Top Features + LassoCV (Log Target) 预测流程完成 ---")

开始执行 Rent 预测流程 (LassoCV)...
输出文件夹 'LeafEncoding_Combined_Lasso_Prediction_Rent_Log' 已创建或已存在。

--- 从 'Rent_Capped_Aggregated_Data' 加载 Rent 数据 ---
Rent 数据加载成功。
  训练集形状: (98899, 159)
  测试集形状: (9773, 158)

--- 转换布尔列 (Rent) 为整数 (1/0) ---
  转换 train_rent_df 中的 39 个布尔列...
  转换 test_rent_df 中的 38 个布尔列...
--- 布尔列转换完成 ---

--- 准备 Rent 训练和测试数据 ---
  已对 Rent 目标变量 'Price' 应用 log1p 转换。
  对齐后 Rent 特征形状: Train=(98899, 158), Test=(9773, 158)
  使用中位数填充缺失值...
  Rent 数据缺失值填充完成。

--- 训练 Gradient Boosting 模型 (Rent) 以生成叶节点特征 ---
  使用以下参数训练 GBRT: {'n_estimators': 150, 'max_depth': 4, 'min_samples_leaf': 30, 'learning_rate': 0.1, 'subsample': 0.7, 'random_state': 42}
  GBRT 模型 (Rent) 训练完成。
  识别出 Rent Top 10 原始特征: ['板块_平均租金_capped', '面积_数值', '区县_中位数租金_capped', '区县_平均租金_capped', '板块_中位数租金_capped', '板块_平均面积_capped', '物业费_数值', 'lat', 'lon', '楼层_总楼层']

--- 获取叶节点索引 (Rent) ---

--- 对叶节点索引进行 One-Hot 编码 (Rent) ---
  生成了 2191 个 Rent 叶节点二元特征。

--- 组合 Rent 叶节点特征与 Top 10 原始特征 ---
  组合后 Rent 最终特征形状: Train=(98899, 2201), Tes

/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/utils/__init__.py:21: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/utils/__init__.py:21: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/utils/__init__.py:21: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/utils/__init__.py:21: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/utils/__init__.py:21:

  Rent 外部交叉验证得分 (负 Log MAE): -0.1261 +/- 0.0006
  Rent 外部交叉验证得分 (Log MAE): 0.1261

--- 在全部 Rent 训练数据上训练最终的 LassoCV Pipeline ---


/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 240.6854564477767, tolerance: 43.11151017357799
  model = cd_fast.sparse_enet_coordinate_descent(
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 42.46565988398197, tolerance: 38.995882113780596
  model = cd_fast.sparse_enet_coordinate_descent(
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196.88114321627586, tolerance: 45.77225883422149
  model = cd_fast.sparse_enet_coordinate_descent(
/home/lwyxyz/anaconda3/lib/python3.9/site-packages/sklearn/lin

  最终 Rent Pipeline (LassoCV) 训练完成。
  LassoCV (Rent) 找到的最佳 alpha (优化 MSE): 0.000359
  Lasso 模型 (Rent) 使用了 1332 / 2201 个特征。

--- 在完整 Rent 训练集上评估最终 Lasso 模型 (原始租金空间) ---
  Rent 训练集 RMSE (原): 125795.94
  Rent 训练集 MAE  (原): 67804.75
  Rent 训练集 MedAE(原): 36728.21

--- 在 Rent 测试集上进行预测 ---
  Rent 预测完成并已转换回原始租金空间。

--- 创建并保存 Rent 提交文件到 'LeafEncoding_Combined_Lasso_Prediction_Rent_Log' ---
✓ Rent 提交文件已成功保存: LeafEncoding_Combined_Lasso_Prediction_Rent_Log/submission_leaf_combined_lasso_rent_log_mae.csv
  总预测条数: 9773

Rent 提交文件预览 (前5行):
        ID  PredictedPrice
0  2000000    1.694051e+05
1  2000001    4.547180e+05
2  2000002    4.276752e+05
3  2000003    1.835670e+06
4  2000004    1.420077e+06

--- Rent 叶节点编码 + Top Features + LassoCV (Log Target) 预测流程完成 ---
